<a href="https://colab.research.google.com/github/taejjing/kaggle_LANL/blob/master/LANL_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
DATA_DIR = '/content/lanl/'

In [0]:
! mkdir ~/.kaggle
! cp /gdrive/'My Drive'/'Colab Notebooks'/kaggle/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle competitions download -c LANL-Earthquake-Prediction -p {DATA_DIR}
! ls -l {DATA_DIR}

  0% 0.00/33.3k [00:00<?, ?B/s]
100% 33.3k/33.3k [00:00<00:00, 31.8MB/s]
100% 241M/242M [00:01<00:00, 217MB/s]
100% 242M/242M [00:01<00:00, 208MB/s]
100% 2.02G/2.03G [00:14<00:00, 105MB/s]
100% 2.03G/2.03G [00:14<00:00, 147MB/s]
total 2372428
-rw-r--r-- 1 root root      34135 May 16 11:20 sample_submission.csv
-rw-r--r-- 1 root root  253730367 May 16 11:20 test.zip
-rw-r--r-- 1 root root 2175592057 May 16 11:21 train.csv.zip


In [9]:
! unzip {DATA_DIR}train.csv.zip -d {DATA_DIR}
! unzip {DATA_DIR}test.zip -d {DATA_DIR}test
! rm -rf {DATA_DIR}*.zip
! ls -l {DATA_DIR}

Archive:  /content/lanl/train.csv.zip
  inflating: /content/lanl/train.csv  
Archive:  /content/lanl/test.zip
  inflating: /content/lanl/test/seg_430e66.csv  
  inflating: /content/lanl/test/seg_d1a281.csv  
  inflating: /content/lanl/test/seg_05a1b0.csv  
  inflating: /content/lanl/test/seg_f8dd7e.csv  
  inflating: /content/lanl/test/seg_b9bdd7.csv  
  inflating: /content/lanl/test/seg_24c1c9.csv  
  inflating: /content/lanl/test/seg_c5abaa.csv  
  inflating: /content/lanl/test/seg_6262c4.csv  
  inflating: /content/lanl/test/seg_734a88.csv  
  inflating: /content/lanl/test/seg_94a133.csv  
  inflating: /content/lanl/test/seg_d0c280.csv  
  inflating: /content/lanl/test/seg_d36737.csv  
  inflating: /content/lanl/test/seg_f80e44.csv  
  inflating: /content/lanl/test/seg_07c815.csv  
  inflating: /content/lanl/test/seg_7c9433.csv  
  inflating: /content/lanl/test/seg_211486.csv  
  inflating: /content/lanl/test/seg_78ded2.csv  
  inflating: /content/lanl/test/seg_f11f77.csv  
  inflat

https://github.com/Vettejeep/MSDS696-Masters-Final-Project/blob/master/earthquake_model.ipynb

In [0]:
import os
import time
import warnings
import traceback
import numpy as np
import pandas as pd
from scipy import stats
import scipy.signal as sg
import multiprocessing as mp
from scipy.signal import hann
from scipy.signal import hilbert
from scipy.signal import convolve
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from tqdm import tqdm
warnings.filterwarnings("ignore")

In [0]:
SIG_LEN = 150000

NUM_SEG_PER_PROC = 4000
NUM_THREADS = 6

NY_FREQ_IDX = 75000  # the test signals are 150k samples long, Nyquist is thus 75k.
CUTOFF = 18000
MAX_FREQ_IDX = 20000
FREQ_STEP = 2500

TRAIN_DIR = os.path.join(DATA_DIR, 'train')
if not os.path.isdir(TRAIN_DIR) :
    os.mkdir(TRAIN_DIR)
TRAIN_LEN = 629145480

In [0]:
def split_raw_data(i, chunk): 
    print("working ", i)
    chunk.to_csv(os.path.join(TRAIN_DIR, 'raw_data_%d.csv' % i), index=False)

In [12]:
try:
    start = time.time()
    
    max_start_index = TRAIN_LEN - SIG_LEN
    slice_len = max_start_index // 6
    chunks = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'), chunksize=slice_len)
    
    for i, chunk in enumerate(chunks):
        split_raw_data(i, chunk)

finally:
    print("{}s".format(time.time() - start))

working  0
working  1
working  2
working  3
working  4
working  5
working  6
2039.9236369132996s


In [21]:

try:
    start = time.time()
    for i in range(5, 6):
        print("working ", i)
        
        pd0 = pd.read_csv(os.path.join(TRAIN_DIR, "raw_data_{}.csv".format(i)))
        pd1 = pd.read_csv(os.path.join(TRAIN_DIR, "raw_data_{}.csv".format(i+1)))
        pd0 = pd0.append(pd1.loc[:SIG_LEN], ignore_index=True)

        pd0.to_csv(os.path.join(TRAIN_DIR, 'mo_raw_data_%d.csv' % i), index=False)
    
finally:
    print("{}s".format(time.time() - start))

working  5
312.8333923816681s


In [22]:
!gsutil -m cp {TRAIN_DIR}/mo*5.csv gs://tj-kaggle/LANL/data/

Copying file:///content/lanl/train/mo_raw_data_5.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 1 objects/1.6 GiB.                                      
